LSTM trained on gridded forcings for each station

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate
import torch.autograd as autograd
import pickle

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190703-083627'

In [2]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
device = torch.device('cuda' if USE_CUDA else 'cpu')
torch.manual_seed(0)
np.random.seed(0)

writer = SummaryWriter()

CUDA Available


In [3]:
station_data_dict = load_data.load_train_test_lstm()
data_runoff = load_data.load_discharge_gr4j_vic()

../src/load_data.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(os.path.join(dir, f), skiprows=2, skipfooter=1, index_col=False, header=None, names=['runoff'], na_values='-1.2345')
/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
class LSTMRegression(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, batch_size, dropout):
            super(LSTMRegression, self).__init__()
            self.batch_size = batch_size
            self.hidden_dim = hidden_dim
            self.num_layers = num_layers
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout)
            self.linear = nn.Linear(hidden_dim, 1)
            self.hidden = self.init_hidden()
        def init_hidden(self):
            return (torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device),
                    torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device))

        def forward(self, input):
            lstm_out, self.hidden = self.lstm(input, self.hidden)
            return self.linear(lstm_out[-1])

In [5]:
predictions = {}
actuals = {}
seq_len = 5 * 24
train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len)  # first day for which to make a prediction in train set
train_end = '2013-12-31' # last day for which to make a prediction in train set
test_start = '2014-01-01'
test_end = '2014-12-31'
validation_fraction = 0.1

for station in station_data_dict.keys():
    torch.manual_seed(0)
    np.random.seed(0)

    station_rdrs = station_data_dict[station]
    station_runoff = data_runoff[data_runoff['station'] == station].set_index('date')
    if any(station_runoff['runoff'].isna()):
        print('Station', station, 'had NA runoff values. Skipping.')
        continue
    
    num_train_days = len(pd.date_range(train_start, train_end, freq='D'))
    num_test_days = len(pd.date_range(test_start, test_end, freq='D'))
    num_total_days = len(pd.date_range(train_start, test_end, freq='D'))
    
    x = np.zeros((seq_len, num_total_days, station_rdrs.shape[1]))
    for day in range(x.shape[1]):
        # For each day that is to be predicted, cut out a sequence that ends with that day's 23:00:00 and is seq_len long
        x[:,day,:] = station_rdrs[train_start + timedelta(days=day, hours=-seq_len + 24) : train_start + timedelta(hours=23, days=day)]
    
    # Scale training data
    scalers = []  # save scalers to apply them to test data later
    x_train = x[:,:num_train_days,:].copy()
    for i in range(x.shape[2]):
        scalers.append(preprocessing.StandardScaler())
        x_train[:,:,i] = scalers[i].fit_transform(x_train[:,:,i].reshape((-1, 1))).reshape(x_train[:,:,i].shape)
    x_train = torch.from_numpy(x_train).float().to(device)
    y_train = torch.from_numpy(station_runoff.loc[train_start:train_end, 'runoff'].to_numpy()).float().to(device)
    
    # Get validation split
    num_validation_samples = int(x_train.shape[1] * validation_fraction)
    validation_indices = np.random.choice(range(x_train.shape[1]), size=num_validation_samples)
    shuffle_indices = np.arange(x_train.shape[1])
    np.random.shuffle(shuffle_indices)
    x_train = x_train[:,shuffle_indices,:]
    y_train = y_train[shuffle_indices]
    x_val, x_train = x_train[:,:num_validation_samples,:], x_train[:,num_validation_samples:,:]
    y_val, y_train = y_train[:num_validation_samples], y_train[num_validation_samples:]
    print('Shapes: x_train {}, y_train {}, x_val {}, y_val {}'.format(x_train.shape, y_train.shape, x_val.shape, y_val.shape))
    
    # Train model
    learning_rate = 2e-3
    patience = 100
    min_improvement = 0.05
    best_loss_model = (-1, np.inf, None)
    
    # Prepare model
    H = 20
    batch_size = 5
    lstm_layers = 1
    dropout = 0
    weight_decay = 2e-5
    model = LSTMRegression(station_rdrs.shape[1], H, lstm_layers, batch_size, dropout).to(device)
    loss_fn = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    for epoch in range(300):
        epoch_losses = []
        
        shuffle_indices = np.arange(x_train.shape[1])
        np.random.shuffle(shuffle_indices)
        x_train = x_train[:,shuffle_indices,:]
        y_train = y_train[shuffle_indices]

        model.train()
        for i in range(x_train.shape[1] // batch_size):
            model.hidden = model.init_hidden()
            y_pred = model(x_train[:,i*batch_size : (i+1)*batch_size,:])
    
            loss = loss_fn(y_pred, y_train[i*batch_size : (i+1)*batch_size].reshape((batch_size,1))).to(device)
            epoch_losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        epoch_loss = np.array(epoch_losses).mean()
        print('Epoch', epoch, 'mean train loss:\t{}'.format(epoch_loss))
        writer.add_scalar('loss_' + station, epoch_loss, epoch)
        
        # eval on validation split
        model.eval()
        val_pred = pd.Series()
        for i in range(x_val.shape[1] // batch_size):
            batch_pred = model(x_val[:,i*batch_size : (i+1)*batch_size,:]).detach().cpu().numpy().reshape(batch_size)
            val_pred = val_pred.append(pd.Series(batch_pred))
        val_nse, val_mse = evaluate.evaluate_daily(station, val_pred, pd.Series(y_val.cpu().numpy().flatten())[:val_pred.shape[0]])
        print('Epoch {} mean val mse:    \t{},\tnse: {}'.format(epoch, val_mse, val_nse))
        writer.add_scalar('loss_eval_' + station, val_mse, epoch)
        
        if val_mse < best_loss_model[1] - min_improvement:
            best_loss_model = (epoch, val_mse, model.state_dict())  # new best model
        elif epoch > best_loss_model[0] + patience:
            print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
            break
    
    print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
    model.load_state_dict(best_loss_model[2])
    load_data.pickle_model('LSTM_VIC', model, station, time_stamp)
    model.eval()
    
    # scale test data
    x_test = x[:,num_train_days:num_train_days+num_test_days,:].copy()
    for i in range(x.shape[2]):
        x_test[:,:,i] = scalers[i].transform(x_test[:,:,i].reshape((-1, 1))).reshape(x_test[:,:,i].shape)
    print('x_test shape: {}'.format(x_test.shape))
    # if batch size doesn't align with number of samples, add dummies to the last batch
    if x_test.shape[1] % batch_size != 0:
        x_test = np.concatenate([x_test, np.zeros((x_test.shape[0], batch_size - (x_test.shape[1] % batch_size), x_test.shape[2]))], axis=1)
        print('Appended dummy entries to x_test. New shape: {}'.format(x_test.shape))
    
    # Predict
    x_test = torch.from_numpy(x_test).float().to(device)
    predict = station_runoff[test_start:test_end].copy()
    predict['runoff'] = np.nan
    pred_array = np.array([])
    print('Predicting')
    for i in range(x_test.shape[1] // batch_size):
        pred_array = np.concatenate([pred_array, model(x_test[:,i*batch_size : (i+1)*batch_size,:]).detach().cpu().numpy().reshape(batch_size)])
    predict['runoff'] = pred_array[:predict.shape[0]]  # ignore dummies
    
    predictions[station] = predict
    actuals[station] = station_runoff['runoff'].loc[test_start:test_end]    

Shapes: x_train torch.Size([120, 1311, 108]), y_train torch.Size([1311]), x_val torch.Size([120, 145, 108]), y_val torch.Size([145])
Epoch 0 mean train loss:	351.44016656484314
Epoch 0 mean val mse:    	219.7593536376953,	nse: 0.0030008452815286635
Epoch 1 mean train loss:	288.22527204397073
Epoch 1 mean val mse:    	200.15975952148438,	nse: 0.09191980184125115
Epoch 2 mean train loss:	275.0847383225692
Epoch 2 mean val mse:    	195.93031311035156,	nse: 0.11110783295468574
Epoch 3 mean train loss:	264.5125356571365
Epoch 3 mean val mse:    	187.32257080078125,	nse: 0.15015929669674422
Epoch 4 mean train loss:	257.0659980578277
Epoch 4 mean val mse:    	188.29039001464844,	nse: 0.1457684995417532
Epoch 5 mean train loss:	251.9712261707728
Epoch 5 mean val mse:    	185.90553283691406,	nse: 0.1565879718630565
Epoch 6 mean train loss:	239.21848111616745
Epoch 6 mean val mse:    	174.69766235351562,	nse: 0.20743559557777058
Epoch 7 mean train loss:	225.02904286366382
Epoch 7 mean val mse:  

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Shapes: x_train torch.Size([120, 1311, 76]), y_train torch.Size([1311]), x_val torch.Size([120, 145, 76]), y_val torch.Size([145])
Epoch 0 mean train loss:	176.4659547660187
Epoch 0 mean val mse:    	169.0301513671875,	nse: 0.0644134062702334
Epoch 1 mean train loss:	158.06495311844895
Epoch 1 mean val mse:    	159.62667846679688,	nse: 0.11646174964875544
Epoch 2 mean train loss:	148.12753754035208
Epoch 2 mean val mse:    	150.01751708984375,	nse: 0.1696488947420708
Epoch 3 mean train loss:	137.10552190164574
Epoch 3 mean val mse:    	146.6244354248047,	nse: 0.18842969029165213
Epoch 4 mean train loss:	129.23145062755083
Epoch 4 mean val mse:    	142.7267608642578,	nse: 0.21000336484596505
Epoch 5 mean train loss:	122.87344654205874
Epoch 5 mean val mse:    	136.30369567871094,	nse: 0.24555529889918448
Epoch 6 mean train loss:	115.8405862891947
Epoch 6 mean val mse:    	137.9257049560547,	nse: 0.2365774323837262
Epoch 7 mean train loss:	112.78292711124621
Epoch 7 mean val mse:    	132

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Shapes: x_train torch.Size([120, 1311, 60]), y_train torch.Size([1311]), x_val torch.Size([120, 145, 60]), y_val torch.Size([145])
Epoch 0 mean train loss:	101.82514386132132
Epoch 0 mean val mse:    	80.40302276611328,	nse: 0.07903729520113167
Epoch 1 mean train loss:	90.29615338601684
Epoch 1 mean val mse:    	74.89640045166016,	nse: 0.14211197729363234
Epoch 2 mean train loss:	84.80098688494385
Epoch 2 mean val mse:    	67.41207122802734,	nse: 0.22784008153298296
Epoch 3 mean train loss:	79.22250551736082
Epoch 3 mean val mse:    	63.57881164550781,	nse: 0.2717473831831363
Epoch 4 mean train loss:	76.86534476581649
Epoch 4 mean val mse:    	69.39266967773438,	nse: 0.20515350857086478
Epoch 5 mean train loss:	70.86734775766617
Epoch 5 mean val mse:    	58.73984909057617,	nse: 0.3271745123028734
Epoch 6 mean train loss:	66.91967216853536
Epoch 6 mean val mse:    	62.606998443603516,	nse: 0.28287889208249517
Epoch 7 mean train loss:	61.685312505555515
Epoch 7 mean val mse:    	56.80646

...

Shapes: x_train torch.Size([120, 1311, 60]), y_train torch.Size([1311]), x_val torch.Size([120, 145, 60]), y_val torch.Size([145])
Epoch 0 mean train loss:	136.9566311585994
Epoch 0 mean val mse:    	159.15826416015625,	nse: -0.01218519968274867
Epoch 1 mean train loss:	123.68791689483434
Epoch 1 mean val mse:    	163.53781127929688,	nse: -0.040037355618534054
Epoch 2 mean train loss:	116.87142896128975
Epoch 2 mean val mse:    	149.832763671875,	nse: 0.04712158783964049
Epoch 3 mean train loss:	109.60191124745907
Epoch 3 mean val mse:    	149.30706787109375,	nse: 0.050464705635494655
Epoch 4 mean train loss:	103.85028283036392
Epoch 4 mean val mse:    	153.91900634765625,	nse: 0.021134548654590635
Epoch 5 mean train loss:	97.07471846464937
Epoch 5 mean val mse:    	140.2860565185547,	nse: 0.10783489108204702
Epoch 6 mean train loss:	88.49824930750232
Epoch 6 mean val mse:    	140.40579223632812,	nse: 0.1070733983438179
Epoch 7 mean train loss:	80.5886308249399
Epoch 7 mean val mse:   

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Shapes: x_train torch.Size([120, 1311, 220]), y_train torch.Size([1311]), x_val torch.Size([120, 145, 220]), y_val torch.Size([145])
Epoch 0 mean train loss:	10156.16276353734
Epoch 0 mean val mse:    	6042.484375,	nse: -0.21249438545341337
Epoch 1 mean train loss:	9389.30274112352
Epoch 1 mean val mse:    	5601.60986328125,	nse: -0.12402786974249902
Epoch 2 mean train loss:	8978.762861251831
Epoch 2 mean val mse:    	5346.40234375,	nse: -0.07281743969098087
Epoch 3 mean train loss:	8708.553404742524
Epoch 3 mean val mse:    	5223.931640625,	nse: -0.04824236757714706
Epoch 4 mean train loss:	8519.757505081992
Epoch 4 mean val mse:    	5203.07763671875,	nse: -0.044057693012430565
Epoch 5 mean train loss:	8336.04760485933
Epoch 5 mean val mse:    	5063.32373046875,	nse: -0.016014498117137954
Epoch 6 mean train loss:	8220.710467826319
Epoch 6 mean val mse:    	5164.4052734375,	nse: -0.03629770272199573
Epoch 7 mean train loss:	8007.4604890546725
Epoch 7 mean val mse:    	4968.86474609375,

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


x_test shape: (120, 365, 220)
Predicting
Shapes: x_train torch.Size([120, 1311, 92]), y_train torch.Size([1311]), x_val torch.Size([120, 145, 92]), y_val torch.Size([145])
Epoch 0 mean train loss:	1275.8093306203834
Epoch 0 mean val mse:    	384.80218505859375,	nse: -0.008595802407941111
Epoch 1 mean train loss:	1186.258789749091
Epoch 1 mean val mse:    	371.8889465332031,	nse: 0.02525068756783544
Epoch 2 mean train loss:	1155.0566224319334
Epoch 2 mean val mse:    	365.98565673828125,	nse: 0.04072375234427694
Epoch 3 mean train loss:	1127.028678237027
Epoch 3 mean val mse:    	362.9809875488281,	nse: 0.04859928891951171
Epoch 4 mean train loss:	1100.096148623765
Epoch 4 mean val mse:    	344.5775451660156,	nse: 0.09683604058377349
Epoch 5 mean train loss:	1080.3278109317519
Epoch 5 mean val mse:    	325.7904968261719,	nse: 0.14607828862599037
Epoch 6 mean train loss:	1045.3755365719321
Epoch 6 mean val mse:    	319.2677917480469,	nse: 0.16317485514161922
Epoch 7 mean train loss:	1020

In [6]:
nse_list = []
mse_list = []
for station, predict in predictions.items():
    nse, mse = evaluate.evaluate_daily(station, predict['runoff'], actuals[station], writer=writer)
    nse_list.append(nse)
    mse_list.append(mse)
    
    print(station, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))
print('Median MSE (clipped to 0)', np.median(mse_list), '/ Min', np.min(mse_list), '/ Max', np.max(mse_list))

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


02GA010 	NSE: -0.035617675371497404 	MSE: 512.8875302285987 (clipped to 0)
02GA018 	NSE: 0.019482188948631407 	MSE: 207.17048578375838 (clipped to 0)
02GA038 	NSE: 0.13877285901460445 	MSE: 142.57098661360203 (clipped to 0)
02GA047 	NSE: -0.1461258757084265 	MSE: 92.77269689242644 (clipped to 0)
02GB001 	NSE: -0.009614822705753756 	MSE: 8062.972398952933 (clipped to 0)
02GB007 	NSE: 0.2527416657417514 	MSE: 18.719597008116477 (clipped to 0)
02GC002 	NSE: 0.2055046419348885 	MSE: 50.75381985473933 (clipped to 0)
02GC007 	NSE: 0.03049986342806965 	MSE: 21.69397071073669 (clipped to 0)
02GC010 	NSE: 0.17117663512132786 	MSE: 22.685100785253553 (clipped to 0)
02GC026 	NSE: 0.08729301506696097 	MSE: 95.69784482923914 (clipped to 0)
02GD004 	NSE: 0.08344410465662311 	MSE: 40.49871978079374 (clipped to 0)
02GE007 	NSE: 0.3010697730801575 	MSE: 21.507179546221273 (clipped to 0)
02GG002 	NSE: 0.18295498953618494 	MSE: 149.51017900655336 (clipped to 0)
02GG003 	NSE: -0.017229023802977972 	MSE: 3

In [7]:
writer.close()

In [8]:
load_data.pickle_results('LSTM_VIC', (predictions, actuals), time_stamp)

'LSTM_VIC_20190703-083627.pkl'

In [9]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190703-125551'